In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=32,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p2d = test_batch['fisheye_joints_2d']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img,0,False)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['regressor_dict']['pred_joint']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp616'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp616/0.pth',
 '/workspace/2d_to_3d/apps/exp616/1.pth',
 '/workspace/2d_to_3d/apps/exp616/2.pth',
 '/workspace/2d_to_3d/apps/exp616/3.pth',
 '/workspace/2d_to_3d/apps/exp616/4.pth',
 '/workspace/2d_to_3d/apps/exp616/5.pth',
 '/workspace/2d_to_3d/apps/exp616/6.pth',
 '/workspace/2d_to_3d/apps/exp616/7.pth',
 '/workspace/2d_to_3d/apps/exp616/8.pth',
 '/workspace/2d_to_3d/apps/exp616/9.pth',
 '/workspace/2d_to_3d/apps/exp616/best.pth']

In [4]:
for i,p in enumerate(path_):
	output_path = f'/dataset/data/ours3/baseline616_{i}(heatmapWASP_depthmapWASP_silhouetteWASP_depthmapconv_silconv).json'
	main(model_path=p,output_path=output_path)

01:58:54 [INFO] Main: Starting demo...
  0%|          | 0/2676 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 149.34170532226562, 'UpperStretching': 152.1468505859375, 'Talking': 114.59046936035156, 'Reacting': 125.60941314697266, 'Patting': 132.6208953857422, 'Walking': 143.79925537109375, 'LowerStretching': 158.7571563720703, 'Gesticuling': 122.4810791015625, 'Greeting': 98.0261459350586, 'Gaming': 146.30210876464844}, 'UpperBody': {'All': 110.11471557617188, 'UpperStretching': 118.5779800415039, 'Talking': 103.43595123291016, 'Reacting': 90.92233276367188, 'Patting': 100.10587310791016, 'Walking': 101.97915649414062, 'LowerStretching': 97.70677185058594, 'Gesticuling': 93.81127166748047, 'Greeting': 86.31694793701172, 'Gaming': 96.30802154541016}, 'LowerBody': {'All': 188.56869506835938, 'UpperStretching': 185.71568298339844, 'Talking': 125.74500274658203, 'Reacting': 160.2965087890625, 'Patting': 165.13589477539062, 'Walking': 185.61935424804688, 'LowerStretching': 219.8075408935547, 'Gesticuling': 151.15087890625, 'Greeting': 109.73534393310547, 'Gaming': 196.29621887

 15%|█▍        | 401/2676 [01:17<06:12,  6.11it/s]

{'FullBody': {'All': 149.8065185546875, 'UpperStretching': 152.75775146484375, 'Talking': 110.6195068359375, 'Reacting': 131.8581085205078, 'Patting': 128.95639038085938, 'Walking': 143.7160186767578, 'LowerStretching': 159.56214904785156, 'Gesticuling': 118.8616943359375, 'Greeting': 100.63401794433594, 'Gaming': 145.08013916015625}, 'UpperBody': {'All': 110.22057342529297, 'UpperStretching': 118.68317413330078, 'Talking': 98.89466094970703, 'Reacting': 92.8178482055664, 'Patting': 97.73950958251953, 'Walking': 102.20856475830078, 'LowerStretching': 98.09280395507812, 'Gesticuling': 90.90836334228516, 'Greeting': 87.9468002319336, 'Gaming': 96.47061920166016}, 'LowerBody': {'All': 189.3924560546875, 'UpperStretching': 186.83233642578125, 'Talking': 122.34434509277344, 'Reacting': 170.8983612060547, 'Patting': 160.1732635498047, 'Walking': 185.22348022460938, 'LowerStretching': 221.03150939941406, 'Gesticuling': 146.8150177001953, 'Greeting': 113.32122039794922, 'Gaming': 193.689666748

 22%|██▏       | 601/2676 [01:51<05:47,  5.97it/s]

...

 90%|████████▉ | 2400/2676 [07:05<00:52,  5.26it/s]

{'FullBody': {'All': 70.0366439819336, 'UpperStretching': 70.5716552734375, 'Gesticuling': 57.865089416503906, 'Reacting': 67.92342376708984, 'Walking': 71.9870376586914, 'Gaming': 70.00064086914062, 'LowerStretching': 70.33089447021484, 'Talking': 54.454620361328125, 'Patting': 78.7353744506836, 'Greeting': 50.84220504760742}, 'UpperBody': {'All': 55.31852340698242, 'UpperStretching': 58.66706085205078, 'Gesticuling': 54.17010498046875, 'Reacting': 59.78323745727539, 'Walking': 51.38443374633789, 'Gaming': 57.24716567993164, 'LowerStretching': 46.34035873413086, 'Talking': 49.58776092529297, 'Patting': 59.15888977050781, 'Greeting': 47.96840286254883}, 'LowerBody': {'All': 84.7547836303711, 'UpperStretching': 82.47624206542969, 'Gesticuling': 61.56007385253906, 'Reacting': 76.06360626220703, 'Walking': 92.58964538574219, 'Gaming': 82.7541275024414, 'LowerStretching': 94.3214340209961, 'Talking': 59.32148361206055, 'Patting': 98.3118667602539, 'Greeting': 53.716007232666016}}


 97%|█████████▋| 2601/2676 [07:40<00:13,  5.49it/s]

{'FullBody': {'All': 70.05772399902344, 'UpperStretching': 70.59783935546875, 'Gesticuling': 57.79145431518555, 'Reacting': 68.1555404663086, 'Walking': 72.0707015991211, 'Gaming': 70.04094696044922, 'LowerStretching': 70.33099365234375, 'Talking': 54.31608963012695, 'Patting': 78.25060272216797, 'Greeting': 50.93762969970703}, 'UpperBody': {'All': 55.324607849121094, 'UpperStretching': 58.68122863769531, 'Gesticuling': 54.04286193847656, 'Reacting': 59.814552307128906, 'Walking': 51.40294647216797, 'Gaming': 57.164573669433594, 'LowerStretching': 46.37785339355469, 'Talking': 49.54610824584961, 'Patting': 58.845314025878906, 'Greeting': 48.03411102294922}, 'LowerBody': {'All': 84.79086303710938, 'UpperStretching': 82.51445007324219, 'Gesticuling': 61.54004669189453, 'Reacting': 76.49653625488281, 'Walking': 92.73844909667969, 'Gaming': 82.91732025146484, 'LowerStretching': 94.28413391113281, 'Talking': 59.086082458496094, 'Patting': 97.6558837890625, 'Greeting': 53.841148376464844}}


100%|██████████| 2676/2676 [07:52<00:00,  5.66it/s]
03:25:54 [INFO] Main: Saving evaluation results...
03:25:54 [INFO] Main: Done.
